In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)

In [3]:
page

<Response [200]>

In [4]:
doc = lh.fromstring(page.content)

In [5]:
tr_elements = doc.xpath('//tr')

In [6]:
col = []
i = 0

for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


# CREATING PANDAS DATAFRAME

In [7]:
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    
    if len(T) != 3:
        break
        
    i = 0
    
    for t in T.iterchildren():
        data = t.text_content()
        if i>0:
            try:
                data  = int(data)
            except:
                pass
            
        col[i][1].append(data)
        i+=1    

In [8]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [9]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

In [10]:
df.head()

,Postal Code\n,Borough\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


## Renaming columns

In [11]:
df.columns = ['PostalCode','Borough','Neighbourhood']

In [12]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


## Replacing the '\n' and cleaning

In [13]:
df = df.replace('\n',' ',regex=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Dropping all cells with borough that is not assigned

In [14]:
df.drop(df.index[df['Borough'] == 'Not assigned '], inplace = True)
df = df.reset_index(drop=True)

df.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Combining neighbourhoods based on similar postcodes and Borough

In [15]:
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['PostalCode','Borough','Neighbourhood']
df.head(10)

,PostalCode,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [16]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [17]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [19]:
df.shape

(104, 3)